# SEN122A Statistical Analysis of Choice Behaviour 

## `Session Lab 03 EXTRA:`
## `Combining machine learning and discrete choice models`

**Delft University of Technology**<br>
**Q2 2024**<br>
**Instructor:** Sander van Cranenburgh<br>
**TA:**  Gabriel Nova <br>

### `Instructions`

**Lab sessions aim to:**<br>
* Show and reinforce how models and ideas presented in class are put to practice.<br>
* Help you gather hands-on machine learning skills.<br>

**Lab sessions are:**<br>
* Learning environments where you work with Jupyter notebooks and where you can get support from TAs and fellow students.<br> 
* Not graded and do not have to be submitted. 

### `Application: Modelling neighbourhood choices`

In this lab session, we will use neighbourhood location choice data of lab session 1: Stated Choice (SC) data, which was collected between 2017 and 2018 in four European cities: Hanover, Mainz, Bern, and Zurich. During this lab session, you will train a neural network and a hybrid choices model to uncover people's preferences over residential location choice attributes, such as the distance to the city centre and the share of foreigners in their neighbourhood. 

![sc](data/sc_experiment.png)


This time the emphesize is on the balance that the researcher must strike between behavioural rigour and model fit.
To do so, in this lab session you will (1) develop a multilayer perceptron model and (2) build a hybrid choice models.

**`Learning objectives lab session 03`**

After completing the following lab session you will be able to:
* Train a hybrid choice model, using PyTorch
* Strike a balance between behavioural rigour and model fit
* Reflect on the strength and weaknesses of both data and theory-driven modelling approaches


**`This lab consists of 1 part and has 2 exercises`**

**Part 1**: The L-MNL model
- Excerise 1: "Features in MLP of LMNL model"
- Excerise 2: "Forecasting using L-MNL model"

#### `Import packages`

In [ ]:
# In this environment Biogeme is not available, but the outputs of the linear-adittive RUM-MNL model have been kept. 
# Please, do not uncomment the following lines if you are running this code in an environment where Biogeme is not available.
#import biogeme.database as db
#import biogeme.biogeme as bio
#from biogeme import models
#from biogeme.expressions import Beta, Variable, log
biogeme_available = False

# Import required Python packages and modules
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# Scikit-learn
from sklearn.preprocessing import StandardScaler

#Pytorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Import custom functions
from train_functions import show_loss_plot, print_model_summary, evaluate_lmnl

# Pandas setting to show all columns when displaying a pandas dataframe
pd.set_option('display.max_columns', None)

## Part 1: The L-MNL model

### `1. Load the data set prepared in Lab 03` <br>

In [ ]:
# Create that path to the data file
data_path = Path(f'data/choice_data_cleaned_lab3E.dat')

# Load mode choice data into a pandas DataFrame
df = pd.read_csv(data_path, sep='\t')

# Define the relevant features of the alternatives
features_alt =   ['STORES1', 'TRANSPORT1', 'CITY1', 'NOISE1', 'GREEN1', 'FOREIGN1', 
                  'STORES2', 'TRANSPORT2', 'CITY2', 'NOISE2', 'GREEN2', 'FOREIGN2',
                  'STORES3', 'TRANSPORT3', 'CITY3', 'NOISE3', 'GREEN3', 'FOREIGN3']

# Define the relevant socio-economic variables
features_socio = ['AGE_2','AGE_3','WOMAN_1','HOMEOWNER_1','CAROWNER_1','RESPCITY_2','RESPCITY_3','RESPCITY_4','JOB_1','NONWESTERN_1','WESTERN_1']

# Show the first few rows of the dataframe
df.head()

In [ ]:
# Reconstruct the test and training sets
df_train = df.loc[df['train_set'] == True,:].copy()
df_test  = df.loc[df['test_set'] == True,:].copy()

# Number of observations in the training and test sets
num_obs_train = len(df_train)
num_obs_test  = len(df_test)

print(f'Number of individuals in the df_train and df_test: \t{df_train.ID.nunique()}  {df_test.ID.nunique()} ')
print(f'Number of observations in the df_train and df_test: \t{len(df_train)} {len(df_test)} ')

To efficiently train MLPs it strongly recommended to scale (a.k.a. normalise) the features. We use sk-learn's 'StandardScaler'. 

In [ ]:
# Create X_train and X_test
x_train = df_train[features_alt + features_socio]
x_test  = df_test [features_alt + features_socio]

# Initialize the scaler object
scaler = StandardScaler()

# Fit and transform the training data
scaler = scaler.fit(df[features_alt + features_socio])

# Apply the fitted scaler to the  data sets
x_train_scaled = pd.DataFrame(scaler.transform(df_train[features_alt + features_socio]), columns=[features_alt + features_socio])
x_test_scaled =  pd.DataFrame(scaler.transform(df_test [features_alt + features_socio]), columns=[features_alt + features_socio]) # Apply the fitted scaler to the test data

print('Shape of x_train', x_train_scaled.shape)
print('Shape of x_test', x_test_scaled.shape)

# Create the target values
# Y must be a dummy coded array
y_train_dummy = pd.get_dummies(df_train['CHOICE']).values.astype(int)
y_test_dummy = pd.get_dummies(df_test['CHOICE']).values.astype(int)

### `The L-MNL model `

Next, we will build and train a hybrid model, namely the L-MNL model. This model combines an interpretable part following the assumptions of the MNL model and a neural network that processes the rest of the variables that we do not want to interpret. More specifically, we want the **flexibility** of the MLP but still want to get out an **estimate of the willingness to pay to reduce the walking distance to the grocery stores, expressed in terms of distance to the public transport**.

The figure below conceptually shows this model:<br>

<p align="center">
<img width="600" src="assets/hybrid_model.png">
</p>

`i. Define the L-MNL model`<br>
To create the L-MNL model,we create a new class using PyTorch's nn.Module. <br>
* The MNL part is implemented with separate linear layers for features (transport and foreign), allowing the utility to be calculated for each alternative. <br>
* The MLP part consists of two hidden layers (linear1 and linear2) with a given number of neurons.<br>
* The forward function takes `x_mnl` and `x_mlp` as inputs. x_mlp is passed through the network layers, while applying activation functions. It outputs the utilities of each alternative `V_MLP`. x_mnl is used to compute utilities in a linear-additive fashion. 
* Finally, the utilities of boths parts are summed.

In [ ]:
class LMNL(nn.Module):
    def __init__(self, input_size_mlp, hidden_size1, hidden_size2,output_size):
        super(LMNL,self).__init__()

        # Create betas for the MNL part
        self.B_transport = nn.Linear(1, 1, bias=False )
        self.B_stores   = nn.Linear(1, 1, bias=False )
                                 
        # Create the hidden layers for the MLP part
        self.linear1 = nn.Linear(input_size_mlp, hidden_size1, bias=False)
        self.linear2 = nn.Linear(hidden_size1,   hidden_size2, bias=False) 
        self.linear3 = nn.Linear(hidden_size2,   output_size,  bias=False) 

    def forward(self, X_MNL, X_MLP):
        
        # Utility functions for the MNL part 
        V_A = self.B_transport(X_MNL[:,0].unsqueeze(1))  + self.B_stores(X_MNL[:,1].unsqueeze(1))
        V_B = self.B_transport(X_MNL[:,2].unsqueeze(1))  + self.B_stores(X_MNL[:,3].unsqueeze(1))
        V_C = self.B_transport(X_MNL[:,4].unsqueeze(1))  + self.B_stores(X_MNL[:,5].unsqueeze(1))
                     
        # Concatenating tensors to maintain the output dimension
        V_MNL = torch.cat((V_A, V_B, V_C), dim=1)

        # The MLP part
        X_MLP = torch.tanh(self.linear1(X_MLP)) # tanh activation function for the 1st layer
        X_MLP = torch.tanh(self.linear2(X_MLP)) # tanh activation function for the 2nd layer
        V_MLP = self.linear3(X_MLP)             # linear activation function for the output layer

        # Sum the utilities from the MNL and MLP parts
        V = V_MNL + V_MLP
        return V

`ii. Prepare the data for training the L-MNL model`<br>
We need to split the features going into the MLP and MNL parts. Because we want to compute the WTP of stores over transport. Therefore, the features related to transport and stores go into the MNL part, while the other features, including socio-demographics, go into the MLP part. **Importantly**, the MNL features must NOT be scaled.

In [ ]:
# Create dataframe with features for MNL part
features_mnl = ['TRANSPORT1', 'STORES1','TRANSPORT2', 'STORES2','TRANSPORT3', 'STORES3']
x_mnl_train = x_train[features_mnl]
x_mnl_test  = x_test[features_mnl]

print('Shape of x_mnl_train\t', x_mnl_train.shape)
print('Shape of x_mnl_test\t', x_mnl_test.shape)

# Create dataframe with features for MLP part
features_mlp_alt = ['FOREIGN1', 'CITY1', 'NOISE1', 'GREEN1', 
                    'FOREIGN2', 'CITY2', 'NOISE2', 'GREEN2', 
                    'FOREIGN3', 'CITY3', 'NOISE3', 'GREEN3']


features_socio = ['RESPCITY_2','RESPCITY_3','RESPCITY_4']
x_mlp_train = x_train_scaled[features_mlp_alt + features_socio]
x_mlp_test  = x_test_scaled[features_mlp_alt + features_socio]

print('\nShape of x_mlp_train\t', x_mlp_train.shape)
print('Shape of x_mlp_test\t', x_mlp_test.shape)

Convert data to PyTorch tensors

In [ ]:
# train
x_mnl_train_tensor = torch.tensor(x_mnl_train.values, dtype=torch.float)
x_mlp_train_tensor = torch.tensor(x_mlp_train.values, dtype=torch.float)
y_train_dummy_tensor = torch.tensor(y_train_dummy,    dtype=torch.float)

# test
x_mnl_test_tensor = torch.tensor(x_mnl_test.values, dtype=torch.float)
x_mlp_test_tensor = torch.tensor(x_mlp_test.values, dtype=torch.float)
y_test_dummy_tensor = torch.tensor(y_test_dummy,    dtype=torch.float)

print('Size of the X_mnl_train tensor = ', x_mnl_train_tensor.size())
print('Size of the X_mlp_train tensor = ', x_mlp_train_tensor.size())
print('\nSize of the X_mnl_test tensor  = ', x_mnl_test_tensor.size())
print('Size of the X_mlp_test tensor  = ', x_mlp_test_tensor.size())

Create PyTorch DataLoaders

In [ ]:
# Create a DataLoader for the train set
dataset_train_lmnl = TensorDataset(x_mnl_train_tensor, x_mlp_train_tensor, y_train_dummy_tensor)
train_loader_lmnl = DataLoader(dataset_train_lmnl, batch_size=250, shuffle=True)

# Create a DataLoader for the test set
dataset_test_lmnl = TensorDataset(x_mnl_test_tensor, x_mlp_test_tensor, y_test_dummy_tensor)
test_loader_lmnl = DataLoader(dataset_test_lmnl, batch_size=len(x_mnl_test_tensor), shuffle=False)

`iv. Create the L-MNL model object`<br>

In [ ]:
# Define the dimensions
input_size_mnl = x_mnl_train_tensor.size()[1]  # Number of input features
input_size_mlp = x_mlp_train_tensor.size()[1]  # Number of input features
hidden_size1 = 10  # Number of units in first hidden layer
hidden_size2 = 10  # Number of units in second hidden layer
output_size = 3    # Number of output classes
print(f' input_size_mnl = {input_size_mnl}, input_size_mlp = {input_size_mlp}, hidden_size1 = {hidden_size1}, hidden_size2 = {hidden_size2}, output_size = {output_size}')

In [ ]:
# Invoke the L-MNL model
model = LMNL(input_size_mlp, hidden_size1, hidden_size2, output_size)

# Print the model architecture
print_model_summary(model)

`v. Train the L-MNL model`<br>
Finally, we are ready to train the L-MNL model!

In [ ]:
# Define training settings
nEpoch = 1000  # Set the number of epochs
lr = 0.0001  # Set the learning rate
status = 10  # Print status every 'status' epochs
patience = 5  # Number of epochs to wait before early stopping

In [ ]:
# Define the loss function and optimizer
loss_fn = nn.CrossEntropyLoss(reduction='sum')
optimizer = optim.Adam(params=model.parameters(), lr=lr,  betas=(0.9, 0.999), eps=1e-07, weight_decay=0)

# Define lists to store losses
train_losses = []
test_losses = []

# Initialize early stopping counter 
counter = 0
best_loss = np.inf  # Set initial loss to infinity

# Training loop
for epoch in range(nEpoch):
    running_loss = 0.0
    for x_mnl, x_mlp, labels in train_loader_lmnl:
        
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(x_mnl,x_mlp)   

        # Compute the loss
        loss = loss_fn(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Update running loss
        running_loss += loss.item() # Log-likelihood on the training set

    # Evaluate on the test set
    test_loss = evaluate_lmnl(model, loss_fn, test_loader_lmnl)

    # Print status every 'status' epochs or epoch 0
    if epoch == 0 or (epoch + 1) % status == 0:
        print(f'Epoch [{epoch+1:5.0f}/{nEpoch}], Train Loss: {running_loss:0.3f}, Test Loss: {test_loss:0.3f}')

    # Store losses
    train_losses.append(running_loss)
    test_losses.append(test_loss)

    # Implement early stopping
    if test_loss < best_loss:
        best_loss = test_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print(f'Early stopping at epoch {epoch+1}')
            break
        
print(f'\nTraining finished.\tTrain Loss: {running_loss:0.3f}, Test Loss: {test_loss:0.3f}')
show_loss_plot(train_losses, test_losses, num_obs_train, num_obs_test)

In [ ]:
# Compute the Willingness to pay to reduce the distance to the grovery store by one minute
LL_LMNL = best_loss
B_transport_LMNL = model.B_transport.weight.data.item()
B_stores_LMNL = model.B_stores.weight.data.item()

WTP_LMNL = B_stores_LMNL/B_transport_LMNL
print(f'\nWillingness to pay to reduce one minute of walking time to the grocery store is: {WTP_LMNL:.2f} walking minutes to the public transport')

`Reflections`
1. The L-MNL model attains a model fit that is very close to the fully flexible MLP model
1. The L-MNL model gives insight into the trade-off between distance to the grocery stores and distance to the public transport
1. The results recovered WTP is not stable, it fluctuates over runs. 

## `Exercise 1: Features in MLP of LMNL model`<br>

`A` Try reducing the number of features that are feed into the MLP and assess the impact on the model performance and WTP. <br>
E.g. only use `RESPCITY` and `WOMAN`, as in lab session 1 we saw these variables had a significant impact<br>

In [ ]:
# Your code and answers

## `Exercise 2: Forecasting using L-MNL`<br>

Suppose that a researcher would like to use this model to make forecasts for Zurich (as we did in Lab session 1). However, this is more challenging.<br>

`A` Give 2 reasons why making forecasts using the L-MNL model is less straightforward than using the MNL models<br>
`B` Give 2 arguments in favour of using the L-MNL model for informing the municipality of Zurich about the impacts of the planned redevelopment on the residential demand per neighbourhood.<br>
`C` Suppose that you are hired to assist and advise this researcher. What would you advise in terms of e.g. the specification of the L-MNL model, training, benchmarking, etc.<br>

In [ ]:
# Your code and answers